# Intro

This will be an example of data analysis on data of data analyst jobs from the website glassdoor.
This will be seperated into chapters where various transformations and snippets will be applied/used;

**[Insights](#Insights)** = Insights gathered after all the transformations in the chapters below. If you are curious how all these were generated then read the later chapters to see the **Python code** used.

**[Chapter 1](#Chapter-1)** = Simple transformations of string data into appropriate integer data. 

**[Chapter 2](#Chapter-2)** = Some form of **Regex (Regular Expressions)**, not my forte. 


-------------------------------------------------------------------------------------------------------------------------------

# Insights
<a id='Insights'></a>

-------------------------------------------------------------------------------------------------------------------------------

# Chapter 1
<a id='Chapter 1'></a>



Import necessary Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

Load original data from CSV and view it and start planning transoformations to get it in a useable format.

In [2]:
pd.read_csv('C:/Users/Asus/Desktop/python/Data/jobs.csv')

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),-1,True
1,1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,-1
2,2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace\n3.4,"New York, NY","New York, NY",1001 to 5000 employees,2003,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,GoDaddy,-1
3,3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,4.1,Celerity\n4.1,"New York, NY","McLean, VA",201 to 500 employees,2002,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD),-1,-1
4,4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,3.9,FanDuel\n3.9,"New York, NY","New York, NY",501 to 1000 employees,2009,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$100 to $500 million (USD),DraftKings,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2248,2248,RQS - IHHA - 201900004460 -1q Data Security An...,$78K-$104K (Glassdoor est.),Maintains systems to protect data from unautho...,2.5,"Avacend, Inc.\n2.5","Denver, CO","Alpharetta, GA",51 to 200 employees,-1,Company - Private,Staffing & Outsourcing,Business Services,Unknown / Non-Applicable,-1,-1
2249,2249,Senior Data Analyst (Corporate Audit),$78K-$104K (Glassdoor est.),Position:\nSenior Data Analyst (Corporate Audi...,2.9,Arrow Electronics\n2.9,"Centennial, CO","Centennial, CO",10000+ employees,1935,Company - Public,Wholesale,Business Services,$10+ billion (USD),"Avnet, Ingram Micro, Tech Data",-1
2250,2250,"Technical Business Analyst (SQL, Data analytic...",$78K-$104K (Glassdoor est.),"Title: Technical Business Analyst (SQL, Data a...",-1.0,Spiceorb,"Denver, CO",-1,-1,-1,-1,-1,-1,-1,-1,-1
2251,2251,"Data Analyst 3, Customer Experience",$78K-$104K (Glassdoor est.),Summary\n\nResponsible for working cross-funct...,3.1,Contingent Network Services\n3.1,"Centennial, CO","West Chester, OH",201 to 500 employees,1984,Company - Private,Enterprise Software & Network Solutions,Information Technology,$25 to $50 million (USD),-1,-1


Assign loaded dataframe.

In [3]:
df = pd.read_csv('C:/Users/Asus/Desktop/python/Data/jobs.csv')

Drop unnecessary columns with the keyword inplace so we dont have to assign the new dataframes created.

In [4]:
df.drop('Easy Apply', axis=1, inplace=True)
df.drop('Competitors', axis=1, inplace=True)
df.drop('Headquarters', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)

Convert the 'k' in 'salary estimate' column to thousands and get rid of unncessary characters, then convert range to the median value (remember it is in USD), then convert USD to £'s (today's rate : $1 = £0.76), try and except to capture potential nuls or anything else that isnt what we expect and delete the offending row (Be sure to check the offending rows beforehand! In this case it was null values denoted with a '-1' as shown below in the output).

In [5]:
for j, i in enumerate(df['Salary Estimate']):
        i = i.replace('K', '000')
        i = i.replace('(Glassdoor est.)', '')
        i = i.replace('$', '')
        i = i.strip('')
        indexofdash = i.find('-')
        num1 = i[:indexofdash]
        num2 = i[indexofdash+1:]
        try:
            new = int(((int(num1) + int(num2))/2)*0.76)
            df['Salary Estimate'][j] = new
        except ValueError:
            print(f"row {j} cannot be parsed as an int, it's value is {i}; dropping row")
            df.drop(j)

C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


row 2149 cannot be parsed as an int, it's value is -1; dropping row


Convert column to be of 'integer' type rather than 'object', then use keyword downcast to convert to lowest memory-form of the integer datatype to save memory (within reason for the values in the column, which pandas decides).

In [6]:
df['Salary Estimate'] = pd.to_numeric(df['Salary Estimate'], downcast='integer')

Make new column for age of company (better for machine learning than year founded), does this for all values that have 4 characters else it marks it with a '?' so I can do some further digging into what the values are apart from years

In [7]:
df['Company Age'] = [2020 - i if len(str(i)) == 4 else '?' for i in df['Founded']]

Check the values that were not 4 characters in df['Founded']

In [8]:
values = {}
for j, i in enumerate(df['Company Age']):
    if i == '?':
        if df['Founded'][j] not in values.keys():
            values[df['Founded'][j]] = 1
        else:
            values[df['Founded'][j]] += 1
print(values)

{-1: 660}


All the values that were not what we expected are -1's which denote nulls, since 660 rows is a lot we will set the nulls as the mean of the companies age's, after we are going to drop the 'Founded' column as we no longer need it 

In [9]:
values = [int(i) for i in df['Company Age'] if i != '?']
mean = int((sum(values))/len(values))

for j, i in enumerate(df['Company Age']):
    if i == '?':
        df['Company Age'][j] = mean
        
df.drop('Founded', axis=1, inplace=True)


C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Transform into a downcast integer type to save memory once again

In [10]:
df['Company Age'] = pd.to_numeric(df['Company Age'], downcast='integer')

-------------------------------------------------------------------------------------------------------------------------------

# Chapter 2 
<a id='Chapter 2'></a>

Get the words we want to filter for in job title by first getting all the individual words in the 'job title' column and then filter and keep only the words with 15 occurences, then do some manaual cleaning.
The variable 'counts' contains all the individual words with the corresponding frequency associated with each word in a python  dictionary fashion whilst the 'wordsGreaterThan15' variable contains the words with greater than 15 occurences (minus a few that were not necessary and removed manually) in a python list fashion.
then proceed to create replicas of the words with their 0th element in upercase and another replica with the whole word in uppercase (we do this to account for words with these uppercase letters).

In [15]:
unwantedchar = '[,]./!{}()>?<|\&^*£$%^-_+='
counts = {}
text = ''
wordsGreaterThan15 = []



for j, i in enumerate(df['Job Title']):
    i = i.translate({ord(h): None for h in unwantedchar}) #remove unwanted chars in each value of job title
    i = i.lower() # make each value of job title lowercase
    df['Job Title'][j] = i # assign cleaned up value of Job Title back to relevant row 
    for j in i:
        if j != '' and j != ' ':
            text += j
        else:
            if text not in counts.keys():
                counts[text] = 1
                text = ''
            else:
                counts[text] += 1
                text = ''
sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True) #sort 'counts' with the key with largest value showing first and so forth

# iterate over the keys in 'counts' and check their corresponding value; if greater than 15 append to 'wordsGreaterThan15'

for i in counts.keys():
    if counts[i] >= 15:
        wordsGreaterThan15.append(i)

# delete words which are not useful so we do not to filter for them
        
del wordsGreaterThan15[2]
del wordsGreaterThan15[3]
del wordsGreaterThan15[-1]
del wordsGreaterThan15[8]
del wordsGreaterThan15[10]

# add in a few relevant words

wordsGreaterThan15.append('sr')
wordsGreaterThan15.append('eng')
wordsGreaterThan15.append('engineering')

# add in skill words to filter for, this will be useful later 

skillwords = [] # will be useful for later

wordsGreaterThan15.append('python')
skillwords.append('python')
wordsGreaterThan15.append('r')
skillwords.append('r')
wordsGreaterThan15.append('sas')
skillwords.append('sas')
wordsGreaterThan15.append('sap')
skillwords.append('sap')
wordsGreaterThan15.append('excel')
skillwords.append('excel')
wordsGreaterThan15.append('sql')
skillwords.append('sql')
wordsGreaterThan15.append('tableau')
skillwords.append('tableau')
wordsGreaterThan15.append('power')
skillwords.append('power')
wordsGreaterThan15.append('aws')
skillwords.append('aws')
wordsGreaterThan15.append('azure')
skillwords.append('azure')
wordsGreaterThan15.append('gcp')
skillwords.append('gcp')
wordsGreaterThan15.append('cloud')
skillwords.append('cloud')
    
wordsGreaterThan15 = sorted(wordsGreaterThan15)
print(wordsGreaterThan15)
        

C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


['ANALYST', 'ANALYSTBUSINESS', 'ANALYSTDATA', 'ANALYSTFINANCIAL', 'ANALYSTHEALTHCARE', 'ANALYSTJUNIOR', 'ANALYSTLEAD', 'ANALYSTSENIOR', 'ANALYSTSR', 'ANALYTICS', 'ANALYTICSDATA', 'Analyst', 'Analystbusiness', 'Analystdata', 'Analystfinancial', 'Analysthealthcare', 'Analystjunior', 'Analystlead', 'Analystsenior', 'Analystsr', 'Analytics', 'Analyticsdata', 'BI', 'BUSINESS', 'Bi', 'Business', 'DATA', 'Data', 'FINANCIAL', 'Financial', 'GOVERNANCE', 'Governance', 'HEALTHCARE', 'Healthcare', 'INTELLIGENCE', 'Intelligence', 'LEAD', 'Lead', 'MANAGEMENT', 'MANAGER', 'MASTER', 'Management', 'Manager', 'Master', 'OPERATIONS', 'Operations', 'PYTHON', 'Python', 'QUALITY', 'Quality', 'REPORTING', 'Reporting', 'SCIENCE', 'SECURITY', 'SENIOR', 'SQL', 'SYSTEMS', 'Science', 'Security', 'Senior', 'Sql', 'Systems', 'WAREHOUSE', 'Warehouse', 'analyst', 'analystbusiness', 'analystdata', 'analystfinancial', 'analysthealthcare', 'analystjunior', 'analystlead', 'analystsenior', 'analystsr', 'analytics', 'analy

In [26]:
wordsOfJobTitle = []

for j, i in enumerate(df['Job Title']):
    for 
    
    

data analyst center on immigration and justice cij
quality data analyst
senior data analyst insights  analytics team customer operations
data analyst
reporting data analyst
data analyst
businessdata analyst fpa
data science analyst
data analyst
data analyst merchant health
data analyst
data analyst
data analyst
senior data analyst
investment advisory data analyst
sustainability data analyst
data analyst
clinical data analyst
data programmeranalyst
data analyst
product analyst data science
data science analyst
data analyst  intex developer
data analyst
entry level  jr data analyst
data analyst
data  business intelligence analyst
data analyst product
data analyst entry level
data analyst
data analyst
data analyst
data analyst
data science analyst capital markets
data analyst games
analystassociate global markets credit data analyst
data business analyst
data analyst with exceldax powerbi experience fulltime
data science analystengineer
data analyst
business analyst data platforms
behavio

healthcare data analyst
data analyst
data analyst predictive analytics
healthcare data analyst
data analyst
data analyst
junior data analyst
covid19 data analyst
business data analyst
junior data analyst python
it data analyst power bi
data analyst
local data analyst  clinical
quality data analyst
senior data analyst
ending the hiv epidemic data analyst
senior data analyst
data analyst junior
sr healthcare data analyst
sr data analyst ii
operationsquality data analyst
sr it data analyst
technical data analyst
senior product data analyst
business data analyst
data analyst
data analyst
it data analyst
data analyst i
business data analyst  marketing
businessdata analyst
demand response dr and customer success csm bi and data analyst
data analyst
data development analyst~
data analyst
data analyst ii
junior data analyst
junior data analyst
data analyst
data analyst ii qa
data analyst  revenue assurance
data standards analyst
data analyst
data analyst
business data analyst
data analyst
data